In [ ]:
using LinearAlgebra, SparseArrays
using Symbolics

@variables λ 

In [83]:
function LU_decompose(A₁)
    n = size(A₁, 1)
    T = typeof(A₁)
    L = T(diagm(ones(n)))
    
    ## 徒手LU分解
    for i = 1:n-1
        r1 = A₁[i, :]
        # U[i, :] = r1
        for j = i+1:n    
            f = A₁[j, i] / A₁[i, i]
            L[j, i] = f
            A₁[j, :] .= A₁[j, :] .- (f * r1)
            # 为啥要引入U，这样已经求解完成了
            # L[:, 1] = A₁[j, :]
        end
        # println("i = $i")
        # display(A₁)
    end
    (;L, U=A₁)
end

LU_decompose (generic function with 1 method)

In [76]:
speye(n) = SparseArrays.sparse(I, n, n)

function Base.diff(x::SparseMatrixCSC, d::Integer=1)
  D = x[2:end, :] .- x[1:end-1, :]
  d >= 2 ? diff(D, d - 1) : D
end

function ddmat(x::AbstractVector, d::Integer=2)
  m = length(x)
  if d == 0
    return speye(m)
  else
    dx = x[(d+1):m] - x[1:(m-d)] # bug may here
    V = spdiagm(0 => 1.0 ./ dx)
    ## dx计算存在错误
    return V * diff(ddmat(x, d - 1))
  end
end

n = 8
D = ddmat(1:n)*2
display(D)

M = diagm(ones(n)) + λ * D' * D
# ddmat(ddmat(1:n, 1), 1)

6×8 SparseMatrixCSC{Float64, Int64} with 18 stored entries:
 1.0  -2.0   1.0    ⋅     ⋅     ⋅     ⋅    ⋅ 
  ⋅    1.0  -2.0   1.0    ⋅     ⋅     ⋅    ⋅ 
  ⋅     ⋅    1.0  -2.0   1.0    ⋅     ⋅    ⋅ 
  ⋅     ⋅     ⋅    1.0  -2.0   1.0    ⋅    ⋅ 
  ⋅     ⋅     ⋅     ⋅    1.0  -2.0   1.0   ⋅ 
  ⋅     ⋅     ⋅     ⋅     ⋅    1.0  -2.0  1.0

8×8 Matrix{Num}:
  1.0 + λ  -2.0λ         λ           …   0.0          0.0          0.0
 -2.0λ      1.0 + 5.0λ  -4.0λ            0.0          0.0          0.0
  λ        -4.0λ         1.0 + 6.0λ      0.0          0.0          0.0
  0.0       λ           -4.0λ            λ            0.0          0.0
  0.0       0.0          λ              -4.0λ         λ            0.0
  0.0       0.0          0.0         …   1.0 + 6.0λ  -4.0λ         λ
  0.0       0.0          0.0            -4.0λ         1.0 + 5.0λ  -2.0λ
  0.0       0.0          0.0             λ           -2.0λ         1.0 + λ

In [78]:
U = deepcopy(M)
L = typeof(U)(diagm(ones(n)))

# 仅运行三次，猜测公式的形式
for i = 1:3
    r1 = U[i, :]
    for j = i+1:min(i+2, n)
        f = U[j, i] / U[i, i]
        L[j, i] = f
        U[j, :] .= U[j, :] .- (f * r1)
    end
end
## U
## Elier采用的是LU分解 
display(L)
display(U)


8×8 Matrix{Num}:
      1.0                …  0.0  0.0  0.0  0.0  0.0
    (-2.0λ) / (1.0 + λ)     0.0  0.0  0.0  0.0  0.0
 λ / (1.0 + λ)              0.0  0.0  0.0  0.0  0.0
      0.0                   1.0  0.0  0.0  0.0  0.0
      0.0                   0.0  1.0  0.0  0.0  0.0
      0.0                …  0.0  0.0  1.0  0.0  0.0
      0.0                   0.0  0.0  0.0  1.0  0.0
      0.0                   0.0  0.0  0.0  0.0  1.0

8×8 Matrix{Num}:
 1.0 + λ  …   0.0          0.0          0.0
 0            0.0          0.0          0.0
 0            0.0          0.0          0.0
 0.0          λ            0.0          0.0
 0.0         -4.0λ         λ            0.0
 0.0      …   1.0 + 6.0λ  -4.0λ         λ
 0.0         -4.0λ         1.0 + 5.0λ  -2.0λ
 0.0          λ           -2.0λ         1.0 + λ

In [60]:
# typeof(U)

8×8 Matrix{Num}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0

In [79]:
# U = deepcopy(M)
# L,U = LU_decompose(U)
# L

In [81]:
r = lu(M)
## lu采用for循环的形式，
r.L
# r.U

8×8 Matrix{Num}:
      1                  …  0
    (-2.0λ) / (1.0 + λ)     0
 λ / (1.0 + λ)              0
      0.0                   0
      0.0                   0
      0.0                …  0
      0.0                   0
      0.0                   1